# Red neuronal

In [351]:
import utils.mnist_reader as mnist_reader
import math
import numpy as np

X_train, y_train = mnist_reader.load_mnist('data/fashion', kind='train')

El primer paso es crear una funcion para crear una red neuronal que reciba como parametros la cantidad de neuronas de input, la cantidad de layer, la cantidad de neuronas en las hidden layers y la cantidad de neuronas de output

In [352]:
import random as rd

def initialize_network(n_inputs, n_hidden, n_outputs, l_hidden):
    network = list() 
    for layer in range(l_hidden): 
        hidden_layer = [{'weights':[rd.random() for i in range(n_inputs + 1)]} for i in range(n_hidden)]
        network.append(hidden_layer)
    output_layer = [{'weights':[rd.random() for i in range(n_hidden + 1)]} for i in range(n_outputs)]
    network.append(output_layer)
    return network

# Prueba de crear una red
rd.seed(1)
network = initialize_network(2, 3, 2, 2)
for layer in network:
    print(layer)
    

[{'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614]}, {'weights': [0.2550690257394217, 0.49543508709194095, 0.4494910647887381]}, {'weights': [0.651592972722763, 0.7887233511355132, 0.0938595867742349]}]
[{'weights': [0.02834747652200631, 0.8357651039198697, 0.43276706790505337]}, {'weights': [0.762280082457942, 0.0021060533511106927, 0.4453871940548014]}, {'weights': [0.7215400323407826, 0.22876222127045265, 0.9452706955539223]}]
[{'weights': [0.9014274576114836, 0.030589983033553536, 0.0254458609934608, 0.5414124727934966]}, {'weights': [0.9391491627785106, 0.38120423768821243, 0.21659939713061338, 0.4221165755827173]}]


El siguiente paso es crear una funcion que haga un forward propagate que reciba como parametros la red neuronal y una lista de inputs

In [353]:
# Calcula la activacion de la neurona considerando todos los pesos e inputs que tiene
def activate(weights, inputs):
    activation = weights[-1]
    for i in range(len(weights)-1):
        activation += weights[i] * inputs[i]
    return activation
 
# Transforma la cativacion al output que tendrá la neurona
def transfer(activation):
    return 1.0 / (1.0 + math.exp(-activation))
 
# Realiza el forward propagate de la red neuronal
def forward_propagate(network, row):
    inputs = row
    for layer in network:
        new_inputs = []
        for neuron in layer:
            activation = activate(neuron['weights'], inputs)
            neuron['output'] = transfer(activation)
            new_inputs.append(neuron['output'])
        inputs = new_inputs
    return inputs

# Prueba de forward propagate
rd.seed(1)
network = initialize_network(2, 3, 2, 2)
row = [1,0]
output = forward_propagate(network, row)
print(output)

[0.7766186921595404, 0.8276836706697595]


Lo siguiente es una implementacion del error del backward propagate

In [354]:
# Calculate the derivative of an neuron output
def transfer_derivative(output):
    return output * (1.0 - output)

# Backpropagate error and store in neurons
def backward_propagate_error(network, expected):
    for i in reversed(range(len(network))):
        layer = network[i]
        errors = list()
        if i != len(network)-1:
            for j in range(len(layer)):
                error = 0.0
                for neuron in network[i + 1]:
                    error += (neuron['weights'][j] * neuron['delta'])
                errors.append(error)
        else:
            for j in range(len(layer)):
                neuron = layer[j]
                errors.append(expected[j] - neuron['output'])
        for j in range(len(layer)):
            neuron = layer[j]
            neuron['delta'] = errors[j] * transfer_derivative(neuron['output'])

# Prueba de back propagate
rd.seed(1)
network = initialize_network(2, 3, 2, 2)
row = [1,0]
forward_propagate(network, row)
expected = [0, 1]
backward_propagate_error(network, expected)
for layer in network:
    print(layer)

[{'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614], 'output': 0.7105668883115941, 'delta': 8.949373877119777e-05}, {'weights': [0.2550690257394217, 0.49543508709194095, 0.4494910647887381], 'output': 0.6691980263750579, 'delta': -0.0035442273587705615}, {'weights': [0.651592972722763, 0.7887233511355132, 0.0938595867742349], 'output': 0.678187028346445, 'delta': -0.0016613352092108096}]
[{'weights': [0.02834747652200631, 0.8357651039198697, 0.43276706790505337], 'output': 0.733450902916955, 'delta': -0.01923100377076876}, {'weights': [0.762280082457942, 0.0021060533511106927, 0.4453871940548014], 'output': 0.7287811747842476, 'delta': 0.001037164080907111}, {'weights': [0.7215400323407826, 0.22876222127045265, 0.9452706955539223], 'output': 0.8335585539346707, 'delta': 0.0002628979455481982}]
[{'weights': [0.9014274576114836, 0.030589983033553536, 0.0254458609934608, 0.5414124727934966], 'output': 0.7766186921595404, 'delta': -0.13472944095336908}, {'weights': [0

Ahora que ya se puede realizar el back propagate para una prueba se necesitan actualizar los pesos de cada neurona.
Aqui se define el learning rate el cual controla el cambio del peso para corregir el error

In [355]:
def update_weights(network, row, l_rate):
    for i in range(len(network)):
        inputs = row[:-1]
        if i != 0:
            inputs = [neuron['output'] for neuron in network[i - 1]]
        for neuron in network[i]:
            for j in range(len(inputs)):
                neuron['weights'][j] += l_rate * neuron['delta'] * inputs[j]
            neuron['weights'][-1] += l_rate * neuron['delta']
            

Ya teniendo la funcion de actualizar el peso se puede proceder a tener una funcion para entrenar la red neuronal.
El numero de epoch es la cantidad de veces que se entrenaá la red con los mismos datos de entrenamiento

In [371]:
def train_network(network, train, l_rate, n_epoch, n_outputs):
    for epoch in range(n_epoch):
        for row in train:
            outputs = forward_propagate(network, row)
            expected = [0 for i in range(n_outputs)]
            expected[row[-1].astype(np.int64)] = 1
            backward_propagate_error(network, expected)
            update_weights(network, row, l_rate)

Ya teniendo la funcion para calcular el error del backpropagate y para entrenar la red neuronal ahora tenemos que predecir.
Esto se logrará teniendo una funcion que prediga teniendo la red neuronal y una fila de inputs

In [357]:
def predict(network, row):
	outputs = forward_propagate(network, row)
	return outputs.index(max(outputs))

Esto es todo lo que se necesita para poder formar una funcion que utilice el algoritmo de backpropagation

In [375]:
def back_propagation(train, test, l_rate, n_epoch, n_hidden, l_hidden):
	n_inputs = len(train[0]) - 1
	n_outputs = len(set([row[-1] for row in train]))
	network = initialize_network(n_inputs, n_hidden, 10, l_hidden)
	train_network(network, train, l_rate, n_epoch, 10)
	predictions = list()
	for row in test:
		prediction = predict(network, row)
		predictions.append(prediction)
	return(predictions)

Ahora que ya se tiene el algoritmo de backward propagation es importante mencionar que para hacer que las pruebas sean bien implementadas se utilizará cross validation.

In [359]:
def cross_validation_split(dataset, n_folds):
	dataset_split = list()
	dataset_copy = list(dataset)
	fold_size = int(len(dataset) / n_folds)
	for i in range(n_folds):
		fold = list()
		while len(fold) < fold_size:
			index = rd.randrange(len(dataset_copy))
			fold.append(dataset_copy.pop(index))
		dataset_split.append(fold)
	return dataset_split

Ahora lo unico que nos queda por hacer es normalizar los datos para que puedan tener un valor entre 0 y 1

In [360]:
def normalize_dataset(dataset):
    minmax = list()
    stats = [[min(column), max(column)] for column in zip(*dataset)]
    for row in dataset:
	    for i in range(len(row)-1):
		    row[i] = (row[i] - stats[i][0]) / (stats[i][1] - stats[i][0])

Ahora que ya tenemos el algoritmo de backward propagation y el dataset dividido en n_folds utilizando cross validation tenemos que
integrar cross validation con backward propagation

In [384]:
def accuracy_metric(actual, predicted):
	correct = 0
	for i in range(len(actual)):
		if actual[i] == predicted[i]:
			correct += 1
	return correct / float(len(actual)) * 100.0

def evaluate_algorithm(dataset, algorithm, n_folds, *args):
    folds = cross_validation_split(dataset, n_folds)
    scores = list()
    for fold in folds:
        #train_set = list(folds)
        #train_set.remove(fold)
        train_set = list()
        for foldd in folds:
            if np.array_equal(foldd, fold):
                train_set.append(foldd)
        train_set = sum(train_set, [])
        test_set = list()
        for row in fold:
            row_copy = list(row)
            test_set.append(row_copy)
            row_copy[-1] = None
        predicted = algorithm(train_set, test_set, *args)
        actual = [row[-1] for row in fold]
        accuracy = accuracy_metric(actual, predicted)
        scores.append(accuracy)
    return scores

Conversiones de columnas

In [362]:
# funcion para convertir a float
def str_column_to_float(dataset, column):
	for row in dataset:
		row[column] = row[column].astype(float)

Ahora que ya se tienen todas las piezas del rompecabezas unidas ya se puede probar la red neuronal

In [363]:
# Se une los el resultado esperado al dataset en la ultima columna
y_trans = np.array([y_train]).T
dataset = np.append(X_train.copy().astype(float), y_trans, 1)


In [ ]:
rd.seed(1)
normalize_dataset(dataset)

In [405]:
dataset2 = dataset.copy()[:10000, :]

In [406]:
n_folds = 5
l_rate = 0.3
n_epoch = 1
n_hidden = 300
l_hidden = 1
scores = evaluate_algorithm(dataset2, back_propagation, n_folds, l_rate, n_epoch, n_hidden, l_hidden)
print('Scores: %s' % scores)
print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))

Scores: [10.0, 8.0, 9.85, 9.6, 9.65]
Mean Accuracy: 9.420%
